## 0. Simulate some data and fit an emulator

In [ ]:
import torch
from autoemulate.emulators import GaussianProcess
import pandas as pd

In [ ]:
# df_x = pd.read_csv("./error_data/3d/3_parameters.csv")
# df_y = pd.read_csv("./error_data/3d/output_3_params.csv")
df_x = pd.read_csv("./data/2d/2_parameters.csv")
df_y = pd.read_csv("./data/2d/output_2_params.csv")


In [ ]:
df_x

In [ ]:
df_y

In [ ]:
x = torch.Tensor(df_x.to_numpy()).float()
y = torch.Tensor(df_y.to_numpy()).float()


In [ ]:

torch.manual_seed(0)
idx = torch.randperm(x.shape[0])
# idx

In [ ]:
x = x[idx]
y = y[idx, 1:]
# y = y[idx, :1]
x.shape, y.shape


In [ ]:

x_train, y_train, x_test, y_test = x[:800], y[:800], x[800:], y[800:]


In [ ]:
import matplotlib.pyplot as plt

plt.hist(df_y.iloc[:,1])
plt.show()

In [ ]:

from autoemulate.core.compare import AutoEmulate
from autoemulate.emulators.gaussian_process.kernel import rbf, rbf_plus_constant
from autoemulate.emulators.transformed.base import TransformedEmulator
from autoemulate.transforms import StandardizeTransform, PCATransform

em = TransformedEmulator(
    x_train,
    y_train,
    model=GaussianProcess,
    x_transforms=[StandardizeTransform()],
    y_transforms=[StandardizeTransform()],
    covar_module_fn=rbf_plus_constant,
)

em.fit(x_train, y_train)


In [ ]:
from torchmetrics import R2Score
from autoemulate.core.model_selection import evaluate

evaluate(em.predict(x_train), y_train, metric=R2Score)

In [ ]:
evaluate(em.predict(x_test), y_test, metric=R2Score)


In [ ]:
from autoemulate.core.compare import AutoEmulate

ae = AutoEmulate(x, y, models=[GaussianProcess], log_level="debug", n_iter=2, n_splits=2)


In [ ]:
# plt.scatter(df_x.iloc[:, 0], df_y.iloc[:, 1])

In [ ]:

ae.plot(0)


In [ ]:
df_x

In [ ]:
from autoemulate.core.sensitivity_analysis import SensitivityAnalysis

problem = {
    "num_vars": 2,
    "names": ["target_snr_db", "phase_offset"],
    "bounds": [(-2.0, 30.0), (-1.0, 1.0)],
}

sa = SensitivityAnalysis(emulator=ae.best_result().model, problem=problem)

df_sa = sa.run()

In [ ]:
df_sa

In [ ]:
sa.plot_sobol(df_sa)

## 1. Simple HMC example.

In [ ]:
from autoemulate.calibration.bayes import BayesianCalibration

Start with an "observation" that the GP has been trained on. 

Specifically, we will pretend we have N noisy experimental measurements. We should be able to recover the input parameters.

In [ ]:
idx = -1 # which simulated value to pick
n_obs = 100
noise_scale = 0.05 # set noise as some ratio of the observed value

observations = {"SER": torch.Tensor([0.8]*100)}
observations

In [ ]:
df_x

In [ ]:

parameters_range = {
    "target_snr_db": (-2.0, 30.0),
    "phase_offset": (-1.0, 1.0)
}
# use the simulator parameter_range 
bc = BayesianCalibration(em, parameters_range, observations, 10.0)

Run MCMC (note that below we have set the number of MCMC steps to a very low number, don't expect convergence).

In [ ]:
mcmc = bc.run_mcmc(
    warmup_steps=100, 
    num_samples=10000,
    sampler='nuts',
)

The returned Pyro MCMC object has methods for accessing the generated samples (`mcmc.get_samples()`) or, as shown below, to get their summary statistics.

In [ ]:
mcmc.summary()

## 2. Plotting with Arviz

We have an option to turn the MCMC object into an Arviz object, which can be passed to any of their plotting function.

In [ ]:
import arviz as az

In [ ]:
az_data = bc.to_arviz(mcmc, posterior_predictive=True)

In [ ]:
az.plot_trace(az_data)

In [ ]:

az.plot_pair(az_data, kind='hexbin', colorbar=True)

In [ ]:
az.plot_ppc(az_data, kind='scatter')

In [ ]:
az.plot_autocorr(az_data)